## Import

In [5]:
import pandas as pd
import random
import os
import numpy as np

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier

In [6]:
class CFG:
    SEED = 42

In [7]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(CFG.SEED) # Seed 고정

## Data Load

In [8]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [9]:
!unzip -qq "/content/gdrive/MyDrive/유전체 정보 품종 분류 AI 경진대회.zip"

In [10]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

In [11]:
def get_x_y(df):
    if 'class' in df.columns:
        df_x = df.drop(columns=['id', 'class'])
        df_y = df['class']
        return df_x, df_y
    else:
        df_x = df.drop(columns=['id'])
        return df_x

In [12]:
train_x, train_y = get_x_y(train)
test_x = get_x_y(test)

## Data Pre-processing
### Label-Encoding

In [13]:
class_le = preprocessing.LabelEncoder()
snp_le = preprocessing.LabelEncoder()
snp_col = [f'SNP_{str(x).zfill(2)}' for x in range(1,16)]

In [14]:
snp_data = []
for col in snp_col:
    snp_data += list(train_x[col].values)

In [15]:
train_y = class_le.fit_transform(train_y)
snp_le.fit(snp_data)

LabelEncoder()

In [16]:
for col in train_x.columns:
    if col in snp_col:
        train_x[col] = snp_le.transform(train_x[col])
        test_x[col] = snp_le.transform(test_x[col])

## Model Fit

In [17]:
!pip install -U teddynote

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 7.4 MB/s 
     |████████████████████████████████| 76.6 MB 1.2 MB/s 
     |████████████████████████████████| 81 kB 9.6 MB/s 
     |████████████████████████████████| 210 kB 53.1 MB/s 
     |████████████████████████████████| 78 kB 8.3 MB/s 
     |████████████████████████████████| 147 kB 69.4 MB/s 
     |████████████████████████████████| 50 kB 6.9 MB/s 
     |████████████████████████████████| 112 kB 73.4 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=2a65d8271e227889507da3cd9473b530aef38b663a8dda3249987ab6e7c90679
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 5.1.0
    Uninstalling importlib-metadata-5.1.0:
      Su

In [18]:
# 모듈 import 
from teddynote import models

In [19]:
from lightgbm import LGBMRegressor, LGBMClassifier
from xgboost import XGBRegressor, XGBClassifier
from catboost import CatBoostRegressor, CatBoostClassifier


In [20]:
train_x

,father,mother,gender,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15
0,0,0,0,2,5,1,0,4,2,0,0,5,0,5,1,0,0,0,0
1,0,0,0,2,1,1,2,0,0,1,0,4,0,1,0,4,5,0,0
2,0,0,0,2,5,5,0,4,3,5,0,4,4,1,0,0,0,0,0
3,0,0,0,1,0,5,0,4,0,5,5,0,5,1,5,5,5,0,5
4,0,0,0,2,5,5,3,0,3,0,0,0,0,5,0,0,1,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,0,0,0,2,1,1,0,4,3,1,0,4,0,5,1,4,0,0,0
258,0,0,0,2,5,0,2,0,0,1,4,4,0,1,1,0,1,0,4
259,0,0,0,1,1,5,0,4,0,1,5,4,4,0,5,5,5,2,5
260,0,0,0,1,0,5,0,4,0,5,5,0,4,1,1,4,5,2,5


In [39]:
xgboptuna_binary = models.XGBClassifierOptuna(use_gpu=False)
        
params, preds = xgboptuna_binary.optimize(train_x, 
                                          train_y, 
                                          test_x,
                                          seed = 321,
                                          n_rounds = 100,
                                          eval_metric='f1', n_trials=50)



[I 2022-12-29 10:27:50,162] A new study created in memory with name: no-name-a8ea2c0e-0260-467c-95f7-fb28a6f7f8af
/usr/local/lib/python3.8/dist-packages/teddynote/models.py:33: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  return trial.suggest_loguniform(self.name, self.low, self.high)
/usr/local/lib/python3.8/dist-packages/teddynote/models.py:35: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  return trial.suggest_uniform(self.name, self.low, self.high)
/usr/local/lib/python3.8/dist-packages/teddynote/models.py:33: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/

metric type: f1, score: 0.22489
metric type: f1, score: 0.30237


/usr/local/lib/python3.8/dist-packages/teddynote/models.py:33: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  return trial.suggest_loguniform(self.name, self.low, self.high)
/usr/local/lib/python3.8/dist-packages/teddynote/models.py:35: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  return trial.suggest_uniform(self.name, self.low, self.high)
/usr/local/lib/python3.8/dist-packages/teddynote/models.py:33: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  return trial.suggest_logunifo

metric type: f1, score: 0.19556
metric type: f1, score: 0.39934


/usr/local/lib/python3.8/dist-packages/teddynote/models.py:33: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  return trial.suggest_loguniform(self.name, self.low, self.high)
/usr/local/lib/python3.8/dist-packages/teddynote/models.py:35: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  return trial.suggest_uniform(self.name, self.low, self.high)
[I 2022-12-29 10:27:51,080] Trial 0 finished with value: 0.2671648515035883 and parameters: {'lambda': 0.00021700048030612396, 'alpha': 0.0006069379680421174, 'colsample_bytree': 0.8461094562740125, 'subsample': 0.6442376386586279, 'learning_rate': 0.0007807685100047831, 'n_estimators': 1838, 'max_depth': 9, 'm

...

metric type: f1, score: 0.96186
metric type: f1, score: 0.90597


/usr/local/lib/python3.8/dist-packages/teddynote/models.py:33: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  return trial.suggest_loguniform(self.name, self.low, self.high)
/usr/local/lib/python3.8/dist-packages/teddynote/models.py:35: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  return trial.suggest_uniform(self.name, self.low, self.high)
/usr/local/lib/python3.8/dist-packages/teddynote/models.py:33: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  return trial.suggest_logunifo

metric type: f1, score: 0.90362
metric type: f1, score: 0.89994


/usr/local/lib/python3.8/dist-packages/teddynote/models.py:33: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  return trial.suggest_loguniform(self.name, self.low, self.high)
/usr/local/lib/python3.8/dist-packages/teddynote/models.py:35: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  return trial.suggest_uniform(self.name, self.low, self.high)
/usr/local/lib/python3.8/dist-packages/teddynote/models.py:33: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  return trial.suggest_logunifo

metric type: f1, score: 0.96108
saving model...models/XGBClassifier-0.95358.npy


In [41]:
result = models.load_prediction_from_file('/content/models/XGBClassifier-0.95358.npy')

In [42]:
result

array([0, 1, 2, 1, 0, 1, 2, 1, 0, 0, 2, 1, 1, 0, 1, 1, 0, 1, 1, 2, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 2, 0, 1, 2, 1, 1, 2, 0, 1, 2, 1,
       1, 1, 1, 2, 1, 2, 0, 1, 0, 1, 1, 1, 2, 0, 1, 2, 0, 1, 2, 2, 2, 0,
       2, 0, 0, 1, 1, 1, 0, 0, 2, 1, 2, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1,
       2, 0, 1, 1, 2, 1, 1, 2, 0, 1, 0, 2, 0, 1, 1, 2, 0, 0, 2, 1, 0, 1,
       2, 1, 1, 1, 1, 0, 0, 2, 1, 1, 0, 1, 1, 2, 2, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 2, 0, 0, 1, 0, 0, 0, 2, 1, 1, 1, 0, 1, 2, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 2, 0, 1, 2, 1, 1, 0, 0, 2, 1, 1, 0, 1, 2, 2, 1, 1])

In [43]:
len(result)

175

In [44]:
result_df = pd.DataFrame(result)

In [45]:
result_df

,0
0,0
1,1
2,2
3,1
4,0
...,...
170,1
171,2
172,2
173,1


In [46]:
result_df[0]

0      0
1      1
2      2
3      1
4      0
      ..
170    1
171    2
172    2
173    1
174    1
Name: 0, Length: 175, dtype: int64

In [47]:
for i in range(175):
  if result_df[0][i] == 0:
    result_df[0][i] = 'A'
  if result_df[0][i] == 1:
    result_df[0][i] = 'B'
  if result_df[0][i] == 2:
    result_df[0][i] = 'C'

In [48]:
result_df

,0
0,A
1,B
2,C
3,B
4,A
...,...
170,B
171,C
172,C
173,B


## Submission

In [49]:
submit = pd.read_csv('./sample_submission.csv')

In [53]:
submit['class'] = result_df[0]

In [54]:
submit

,id,class
0,TEST_000,A
1,TEST_001,B
2,TEST_002,C
3,TEST_003,B
4,TEST_004,A
...,...,...
170,TEST_170,B
171,TEST_171,C
172,TEST_172,C
173,TEST_173,B


In [55]:
submit.to_csv('./submit.csv', index=False)